In [ ]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

host_ip = os.getenv('host_ip')
DATABASE = os.getenv('DATABASE')
user_id = os.getenv('user_id')
user_password = os.getenv('user_password')

# MySQl 연결
conn = mysql.connector.connect(
    host = host_ip, #localhost 또는 EC2의 EIP
    user = user_id,
    password = user_password,
    database = DATABASE
)
conn


In [ ]:
# 커서 생성
cursor = conn.cursor(buffered=True)

In [ ]:
# 쿼리 실행
cursor.execute("SELECT * FROM tbAPTPrice;")

# 데이터를 Pandas DataFrame으로 변환
columns = [desc[0] for desc in cursor.description]  # 컬럼명 가져오기
data = cursor.fetchall()  # 데이터 가져오기

df = pd.DataFrame(data, columns=columns)  # DataFrame 생성

# DataFrame 출력
print(df)

# 커넥션 닫기
cursor.close()

In [ ]:
pd_df = df

In [ ]:
pd_df.sort_values(by = ['접수연도', 'idx'], inplace = True)

pd_df.shape, pd_df.dtypes, pd_df.columns

In [ ]:
pd_df.drop_duplicates(subset = '자치구코드')['자치구코드']

In [ ]:
 data = pd_df[pd_df['자치구코드'] == 11260]\
        .groupby(['접수연도','자치구코드'])\
        .mean('물건금액_만원')\
        .reset_index()\
        .rename(columns={'idx': 'mean_building_price'})

data

In [ ]:

# 고유한 '자치구코드' 리스트 가져오기
unique_codes = pd_df['자치구코드'].drop_duplicates()

# subplot 레이아웃 설정
num_plots = len(unique_codes)  # 필요한 subplot 개수
rows = (num_plots // 3) + 1  # 행 개수 (3열로 설정)
cols = 3                     # 열 개수

fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))  # 전체 subplot 크기 조정
axes = axes.flatten()  # Axes를 1차원 배열로 변환 (루프에서 쉽게 접근 가능)

# 각 자치구코드별로 그래프 그리기
for idx, i in enumerate(unique_codes):
    ax = axes[idx]  # 현재 subplot 선택
    
    # 데이터 필터링 및 그래프 그리기
    data = pd_df[pd_df['자치구코드'] == i]\
        .groupby(['자치구코드', '접수연도'])\
        .mean('물건금액_만원')\
        .reset_index()\
        .rename(columns={'idx': 'mean_building_price'})
    
    sns.lineplot(
        data=data,
        x='접수연도',
        y='mean_building_price',
        ax=ax
    )
    
    # 제목 설정
    ax.set_title(f'자치구코드: {i}', fontsize=10)
    
    # 각 데이터 포인트에 값 표시
    for x, y in zip(data['접수연도'], data['mean_building_price']):
        ax.text(x, y, f'{y:.2f}', fontsize=8, color='black', ha='center')

# 남은 subplot 숨기기 (필요 없을 때)
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

# 레이아웃 정리
plt.tight_layout()
plt.show()

In [ ]:

# 고유한 '자치구코드' 리스트 가져오기
unique_codes = pd_df['건물용도'].drop_duplicates()

# subplot 레이아웃 설정
num_plots = len(unique_codes)  # 필요한 subplot 개수
rows = (num_plots // 3) + 1  # 행 개수 (3열로 설정)
cols = 3                     # 열 개수

fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))  # 전체 subplot 크기 조정
axes = axes.flatten()  # Axes를 1차원 배열로 변환 (루프에서 쉽게 접근 가능)

# 각 자치구코드별로 그래프 그리기
for idx, i in enumerate(unique_codes):
    ax = axes[idx]  # 현재 subplot 선택
    
    # 데이터 필터링 및 그래프 그리기
    data = pd_df[pd_df['건물용도'] == i]\
        .groupby(['건물용도', '접수연도'])\
        .mean('물건금액_만원')\
        .reset_index()\
        .rename(columns={'idx': 'mean_building_price'})
    
    sns.lineplot(
        data=data,
        x='접수연도',
        y='mean_building_price',
        ax=ax
    )
    
    # 제목 설정
    ax.set_title(f'건물용도: {i}', fontsize=10)
        
    # 각 데이터 포인트에 값 표시
    for x, y in zip(data['접수연도'], data['mean_building_price']):
        ax.text(x, y, f'{y:.2f}', fontsize=8, color='black', ha='center')

# 남은 subplot 숨기기 (필요 없을 때)
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

# 레이아웃 정리
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 예제 데이터프레임 (pd_df는 이미 정의되어 있다고 가정)

# 고유한 자치구코드와 건축용도 조합 가져오기
unique_combinations = pd_df[['자치구코드', '건물용도']].drop_duplicates()

# subplot 레이아웃 설정
num_plots = len(unique_combinations)  # 필요한 subplot 개수
rows = (num_plots // 3) + 1           # 행 개수 (3열로 설정)
cols = 3                              # 열 개수

fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))  # 전체 subplot 크기 조정
axes = axes.flatten()  # Axes를 1차원 배열로 변환

# 각 자치구와 건축용도 조합별로 그래프 그리기
for idx, (gu, usage) in enumerate(unique_combinations.values):
    ax = axes[idx]  # 현재 subplot 선택
    
    # 데이터 필터링
    data = pd_df[(pd_df['자치구코드'] == gu) & (pd_df['건물용도'] == usage)]\
        .groupby(['자치구코드', '건물용도', '접수연도'])\
        .mean('건물용도')\
        .reset_index()\
        .rename(columns={'idx': 'mean_building_price'})
    
    # 그래프 그리기
    sns.lineplot(
        data=data,
        x='접수연도',
        y='mean_building_price',
        ax=ax
    )
    
    # 제목 설정
    ax.set_title(f'자치구: {gu}, 용도: {usage}', fontsize=10)

    # 각 데이터 포인트에 값 표시
    for x, y in zip(data['접수연도'], data['mean_building_price']):
        ax.text(x, y, f'{y:.2f}', fontsize=8, color='black', ha='center')

# 남은 subplot 숨기기 (필요 없을 때)
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

# 레이아웃 정리
plt.tight_layout()
plt.show()
